In [3]:
#coding=utf-8
import threading
from time import ctime,sleep

def music(func):
    for i in range(2):
        print("I was listening to %s. %s" %(func,ctime()))
        sleep(1)

def move(func):
    for i in range(2):
        print("I was at the %s! %s" %(func,ctime()))
        sleep(5)

threads = []
t1 = threading.Thread(target=music,args=(u'爱情买卖',))
threads.append(t1)
t2 = threading.Thread(target=move,args=(u'阿凡达',))
threads.append(t2)

if __name__ == '__main__':
    for t in threads:
        t.setDaemon(True)
        t.start()

    print("all over %s" % ctime())

I was listening to 爱情买卖. Sun May 23 15:25:56 2021
I was at the 阿凡达! Sun May 23 15:25:56 2021
all over Sun May 23 15:25:56 2021
I was listening to 爱情买卖. Sun May 23 15:25:57 2021
I was at the 阿凡达! Sun May 23 15:26:01 2021


python提供了两个模块来实现多线程thread 和threading ，thread 有一些缺点，在threading 得到了弥补，为了不浪费你和时间，所以我们直接学习threading 就可以了。

继续对上面的例子进行改造，引入threadring来同时播放音乐和视频：

import threading

首先导入threading 模块，这是使用多线程的前提。

threads = []

t1 = threading.Thread(target=music,args=(u'爱情买卖',))

threads.append(t1)

　　创建了threads数组，创建线程t1,使用threading.Thread()方法，在这个方法中调用music方法target=music，args方法对music进行传参。 把创建好的线程t1装到threads数组中。

　　接着以同样的方式创建线程t2，并把t2也装到threads数组。

for t in threads:

　　t.setDaemon(True)

　　t.start()

最后通过for循环遍历数组。（数组被装载了t1和t2两个线程）


setDaemon()

　　setDaemon(True)将线程声明为守护线程，必须在start() 方法调用之前设置，如果不设置为守护线程程序会被无限挂起。子线程启动后，父线程也继续执行下去，当父线程执行完最后一条语句print "all over %s" %ctime()后，没有等待子线程，直接就退出了，同时子线程也一同结束。



start()

开始线程活动。

从执行结果来看，子线程（muisc 、move ）和主线程（print "all over %s" %ctime()）都是同一时间启动，但由于主线程执行完结束，所以导致子线程也终止。 


我们只对上面的程序加了个join()方法，用于等待线程终止。join（）的作用是，在子线程完成运行之前，这个子线程的父线程将一直被阻塞。

　　注意:  join()方法的位置是在for循环外的，也就是说必须等待for循环里的两个进程都结束后，才去执行主进程。

In [5]:
#coding=utf-8
import threading
from time import ctime,sleep

def music(func):
    for i in range(2):
        print("I was listening to %s. %s" %(func,ctime()))
        sleep(4)

def move(func):
    for i in range(2):
        print("I was at the %s! %s" %(func,ctime()))
        sleep(5)

threads = []
t1 = threading.Thread(target=music,args=(u'爱情买卖',))
threads.append(t1)
t2 = threading.Thread(target=move,args=(u'阿凡达',))
threads.append(t2)

if __name__ == '__main__':
    for t in threads:
        t.setDaemon(True)
        t.start()

    for t in threads:
        t.join()

    print("all over %s" % ctime())

I was listening to 爱情买卖. Sun May 23 15:36:05 2021
I was at the 阿凡达! Sun May 23 15:36:05 2021
I was listening to 爱情买卖. Sun May 23 15:36:09 2021
I was at the 阿凡达! Sun May 23 15:36:10 2021
all over Sun May 23 15:36:15 2021


多线程共享全局变量
线程时进程的执行单元，进程时系统分配资源的最小执行单位，所以在同一个进程中的多线程是共享资源的

In [21]:
import time

g_num = 100
def work1():
    global  g_num
    print('Added work1 %s' % (ctime()))

    for i in range(3):
        g_num+=1
    print('in work1 g_num is : %d. %s' % (g_num,ctime()))

def work2():
    global g_num
    print('Added work2 %s' % (ctime()))
    print('in work2 g_num is : %d. %s' % (g_num,ctime()))

if __name__ == '__main__':
    t1 = threading.Thread(target=work1)
    t1.start()
    time.sleep(1)
    t2=threading.Thread(target=work2)
    t2.start()


Added work1 Sun May 23 15:59:29 2021
in work1 g_num is : 103. Sun May 23 15:59:29 2021
Added work2 Sun May 23 15:59:30 2021
in work2 g_num is : 103. Sun May 23 15:59:30 2021


由于线程之间是进行随机调度，并且每个线程可能只执行n条执行之后，当多个线程同时修改同一条数据时可能会出现脏数据，所以出现了线程锁，即同一时刻允许一个线程执行操作。
线程锁用于锁定资源，可以定义多个锁，像下面的代码，当需要独占。
某一个资源时，任何一个锁都可以锁定这个资源，就好比你用不同的锁都可以把这个相同的门锁住一样。
由于线程之间是进行随机调度的，如果有多个线程同时操作一个对象，如果没有很好地保护该对象，会造成程序结果的不可预期，我们因此也称为“线程不安全”。
为了防止上面情况的发生，就出现了互斥锁（Lock）。

为什么加入是打印前一个print


In [20]:
from threading import Lock, Thread
def work():
    global n
    print('Added.')
    lock.acquire()
    temp = n
    time.sleep(0.1)
    n = temp-1
    print('%s. %d' % (n, ctime()))
    lock.release()

if __name__ == '__main__':
    lock = Lock()
    n = 100
    l = []
    for i in range(10):
        p = Thread(target=work)
        l.append(p)
        p.start()
    for p in l:
        p.join()


Added.
Added.
Added.
Added.
Added.
Added.
Added.
Added.
Added.
Added.
99. Sun May 23 15:58:25 2021
98. Sun May 23 15:58:25 2021
97. Sun May 23 15:58:25 2021
96. Sun May 23 15:58:26 2021
95. Sun May 23 15:58:26 2021
94. Sun May 23 15:58:26 2021
93. Sun May 23 15:58:26 2021
92. Sun May 23 15:58:26 2021
91. Sun May 23 15:58:26 2021
90. Sun May 23 15:58:26 2021


递归锁：RLcok类的用法和Lock类一模一样，但它支持嵌套，在多个锁没有释放的时候一般会使用RLock类

In [7]:
import threading
from time import ctime, sleep

def func(lock):
    global gl_num
    print('Added %d, %s' % (gl_num, ctime()))
    lock.acquire()
    gl_num += 1
    sleep(1)
    print('%d, %s' % (gl_num, ctime()))
    lock.release()
    

if __name__ == '__main__':
    gl_num = 0
    lock = threading.RLock()
    for i in range(10):
        t = threading.Thread(target=func,args=(lock,))
        t.start()
        


Added 0, Sun May 23 17:51:52 2021
Added 1, Sun May 23 17:51:52 2021
Added 1, Sun May 23 17:51:52 2021
Added 1, Sun May 23 17:51:52 2021
Added 1, Sun May 23 17:51:52 2021
Added 1, Sun May 23 17:51:52 2021
Added 1, Sun May 23 17:51:52 2021
Added 1, Sun May 23 17:51:52 2021
Added 1, Sun May 23 17:51:52 2021
Added 1, Sun May 23 17:51:52 2021
1, Sun May 23 17:51:52 2021
2, Sun May 23 17:51:52 2021
3, Sun May 23 17:51:53 2021
4, Sun May 23 17:51:53 2021
5, Sun May 23 17:51:53 2021


In [ ]:
import threading
 
#创建一个lock对象
lock = threading.Lock()
 
#初始化共享资源
abce = 0
 
#本线程访问共享资源
lock.acquire()
abce = abce + 1
 
#这个线程访问共享资源会被阻塞
lock.acquire()
abce = abce + 2
lock.release()
 
print(abce)

In [1]:
import threading
 
#创建一个rlock对象
lock = threading.RLock()
 
#初始化共享资源
abce = 0
 
#本线程访问共享资源
lock.acquire()
abce = abce + 1
 
#这个线程尝试访问共享资源
lock.acquire()
abce = abce + 2
lock.release()
 
print(abce)

3


在这里，没有阻止程序中的线程访问共享资源abce。 对于RLock对象锁的每个acquire()，我们需要调用release()一次。

从上面提到的众多程序和解释中，在Python中一个Lock对象和一个RLock对象有很多区别:

locks	
lock对象无法再被其他线程获取，除非持有线程释放
lock对象可被任何线程释放
lock对象不可以被任何线程拥有
对一个对象锁定是很快的


rlocks
rlock对象可以被其他线程多次获取
rlock对象只能被持有的线程释放
rlock对象可以被多个线程拥有
对一个对象加rlock比加lock慢

In [12]:
import threading

lock = threading.RLock()

def f():
  with lock:
    g()
    h()

def g():
  with lock:
    h()
    g_something()

def h():
  with lock:
    h_something()

def g_something():
    print('g')

def h_something():
    print('h')


# Create and run threads as follows
try:
    threading.Thread( target=f ).start()
    threading.Thread( target=f ).start()
    threading.Thread( target=f ).start()
except Exception as e:
    print("Error: unable to start thread")


h
g
h
h
g
h
h
g
h


信号量（Semaphore类）
互斥锁同时只允许一个线程更改数据，而Semaphore是同时允许一定数量的线程更改数据，比如厕所有3个坑，
那最多只允许3个人上厕所，后面的人只能等里面有人出来了才能再进去


有界信号量BoundedSemaphore

BoundedSemaphore在调用release()的时候，会校验一下当前信号量的值，是否会大于初始值（只定义了5个信号量，释放了5次后，还要调用release）的场景，会抛出异常，而 Semaphore在这种场景下，release()的结果只是None,没有返回信号量对象,并不会抛出异常。


In [4]:
import threading
 
sem=threading.BoundedSemaphore(5)
# sem=threading.Semaphore(5)
global num
num=0
def run():
    global num
    sem.acquire()
    num+=1
    if num>0:
        sem.release()
    print(sem.release())
 
for i in range(5):
    threading.Thread(target=run,args=()).start()
print(num)

5
Exception in thread Thread-30:
Traceback (most recent call last):
  File "C:\Users\yorkh\Anaconda3\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\yorkh\Anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-4-1af52b8e95e3>", line 13, in run
    print(sem.release())
  File "C:\Users\yorkh\Anaconda3\lib\threading.py", line 483, in release
    raise ValueError("Semaphore released too many times")
ValueError: Semaphore released too many times

Exception in thread Thread-32:
Traceback (most recent call last):
  File "C:\Users\yorkh\Anaconda3\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\yorkh\Anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-4-1af52b8e95e3>", line 13, in run
    print(sem.release())
  File "C:\Users\yorkh\Anaconda3\lib\threading.py", line 483, in release
    raise ValueError("Semap

In [ ]:
import threading, time

def run(n,semaphore):
    semaphore.acquire()   #加锁
    time.sleep(3)
    print('run the thread:%s\n' % n)
    semaphore.release()    #释放


if __name__== '__main__':
    num=0
    semaphore = threading.BoundedSemaphore(5)   #最多允许5个线程同时运行
    for i in range(22):
        t = threading.Thread(target=run,args=('t-%s' % i,semaphore))
        t.start()
    while threading.active_count() !=1:
        print(threading.active_count())
        pass
    else:
        print('----------all threads done-----------')


python线程的事件用于主线程控制其他线程的执行，事件是一个简单的线程同步对象，其主要提供以下的几个方法：
        clear将flag设置为 False
        set将flag设置为 True
        is_set判断是否设置了flag
        wait会一直监听flag，如果没有检测到flag就一直处于阻塞状态
事件处理的机制：
全局定义了一个Flag，当Flag的值为False，那么event.wait()就会阻塞，
当flag值为True，那么event.wait()便不再阻塞

In [11]:
# import threading, time

# event = threading.Event()
# def lighter():
#     count = 0
#     event.set()         #初始者为绿灯
#     while True:
#         if 5 < count <=10:
#             event.clear()  #红灯，清除标志位
#             print("\33[41;lmred light is on...\033[0m]")
#         elif count > 10:
#             event.set()    #绿灯，设置标志位
#             count = 0
#         else:
#             print('\33[42;lmgreen light is on...\033[0m')

#         time.sleep(1)
#         count += 1


# def car(name):
#     while True:
#         if event.is_set():     #判断是否设置了标志位
#             print('[%s] running.....'%name)
#             time.sleep(1)
#         else:
#             print('[%s] sees red light,waiting...'%name)
#             event.wait()
#             print('[%s] green light is on,start going...'%name)


# startTime = time.time()
# light = threading.Thread(target=lighter,)
# light.start()

# car = threading.Thread(target=car,args=('MINT',))
# car.start()
# endTime = time.time()
# print('用时：',endTime-startTime)



mgreen light is on...
mgreen light is on...
None
mgreen light is on...
[MINT] running.....
用时： 0.007035970687866211
mred light is on...]
mred light is on...]
mgreen light is on...
mgreen light is on...
mgreen light is on...
mgreen light is on...
mgreen light is on...
mgreen light is on...
mgreen light is on...[MINT] sees red light,waiting...

mred light is on...]
mgreen light is on...
[MINT] green light is on,start going...
[MINT] running.....
mgreen light is on...
mgreen light is on...
mgreen light is on...
mgreen light is on...
mgreen light is on...
mgreen light is on...
mgreen light is on...
mred light is on...]
mgreen light is on...
[MINT] sees red light,waiting...mgreen light is on...

mred light is on...]
mgreen light is on...
mgreen light is on...
mgreen light is on...
mred light is on...]
mgreen light is on...
mgreen light is on...
mred light is on...]
mgreen light is on...
mgreen light is on...
mred light is on...]
mgreen light is on...
mred light is on...]
mred light is on..

GIL  全局解释器
        在非python环境中，单核情况下，同时只能有一个任务执行。多核时可以支持多个线程同时执行。但是在python中，无论有多少个核
        同时只能执行一个线程。究其原因，这就是由于GIL的存在导致的。
        GIL的全程是全局解释器，来源是python设计之初的考虑，为了数据安全所做的决定。某个线程想要执行，必须先拿到GIL，我们可以
        把GIL看做是“通行证”，并且在一个python进程之中，GIL只有一个。拿不到线程的通行证，并且在一个python进程中，GIL只有一个，
        拿不到通行证的线程，就不允许进入CPU执行。GIL只在cpython中才有，因为cpython调用的是c语言的原生线程，所以他不能直接操
        作cpu，而只能利用GIL保证同一时间只能有一个线程拿到数据。而在pypy和jpython中是没有GIL的
        python在使用多线程的时候，调用的是c语言的原生过程。

python针对不同类型的代码执行效率也是不同的
        1、CPU密集型代码（各种循环处理、计算等），在这种情况下，由于计算工作多，ticks技术很快就会达到阀值，然后出发GIL的
        释放与再竞争（多个线程来回切换当然是需要消耗资源的），所以python下的多线程对CPU密集型代码并不友好。
        2、IO密集型代码（文件处理、网络爬虫等设计文件读写操作），多线程能够有效提升效率（单线程下有IO操作会进行IO等待，
        造成不必要的时间浪费，而开启多线程能在线程A等待时，自动切换到线程B，可以不浪费CPU的资源，从而能提升程序的执行
        效率）。所以python的多线程对IO密集型代码比较友好。

 主要要看任务的类型，我们把任务分为I/O密集型和计算密集型，而多线程在切换中又分为I/O切换和时间切换。如果任务属于是I/O密集型，
    若不采用多线程，我们在进行I/O操作时，势必要等待前面一个I/O任务完成后面的I/O任务才能进行，在这个等待的过程中，CPU处于等待
    状态，这时如果采用多线程的话，刚好可以切换到进行另一个I/O任务。这样就刚好可以充分利用CPU避免CPU处于闲置状态，提高效率。但是
    如果多线程任务都是计算型，CPU会一直在进行工作，直到一定的时间后采取多线程时间切换的方式进行切换线程，此时CPU一直处于工作状态，
    此种情况下并不能提高性能，相反在切换多线程任务时，可能还会造成时间和资源的浪费，导致效能下降。这就是造成上面两种多线程结果不能的解释。
结论:I/O密集型任务，建议采取多线程，还可以采用多进程+协程的方式(例如:爬虫多采用多线程处理爬取的数据)；对于计算密集型任务，python此时就不适用了。

事件对象

具体什么事件，是程序员定义的，不同的事件使用不同的event对象来控制。每个event对象只有几个简单的接口，set, clear, wait, is_set。https://docs.python.org/3/library/threading.html#event-objects

默认情况下wait是block的，按照python的风格，可以设置一个timeout！这个机制，可以实现一种小于等于某个时间长度的等待。（time.sleep是等待一个固定的时间长度）

event这种多线程同步模型，可以轻松实现多个线程等待同一个event的场景。一个线程set，其它所有线程wait。（我有的时候喜欢用一个变量来代替event，比如只有一个线程用来改变这个变量的值，其它所有线程只是读取值）

In [2]:
import threading
import time


event_xh = threading.Event()


def _worker(index):
    event_xh.wait()
    print(index)


if event_xh.is_set():
    event_xh.clear()


t_list = []
for i in range(10):
    t = threading.Thread(target=_worker, args=(i,))
    t_list.append(t)
    t.start()


time.sleep(10)
event_xh.set()

1
5
8
4
9
3
7
6
2
0


使用threading.Event对象的wait函数，带个timeout参数，可以实现等待时间小于等于某个时间长度。

当代码流程进入event.wait的阻塞时，这个阻塞会在event.set后开始继续，如果event.wait(TIMEOUT)，那么阻塞的等待时间，就是 <= TIMEOUT。

这个机制有些时候能够很好的优化代码逻辑。

In [1]:

import threading
from time import sleep


def test(n, event):
    while not event.isSet():
        print('Thread %s is ready' % n)
        sleep(1)
    event.wait()
    while event.isSet():
        print(threading.current_thread().ident)
        print('Thread %s is running' % n)
        sleep(1)


def main():
    event = threading.Event()
    for i in range(2):
        th = threading.Thread(target=test, args=(i, event))
        th.start()
    sleep(1)
    print('----- event is set -----')
    event.set()
    sleep(3)
    print('----- event is clear -----')
    event.clear()


if __name__ == '__main__':
    main()

Thread 0 is ready
Thread 1 is ready
Thread 0 is ready
----- event is set -----
27580
Thread 1 is running
14540
Thread 0 is running
27580
Thread 1 is running
14540
Thread 0 is running
27580
Thread 1 is running
14540
----- event is clear -----
Thread 0 is running


首先创建了两个子线程，并通过主线程的event对象来控制子线程什么时候开始执行（使用event.wait()方法阻塞子线程）。通过结果可以看出，在主线程sleep 3秒期间（主线程event.set()之前，此时event.isSet()为False），每个子线程都是在打印“...is ready”，并没有继续执行后面的代码。

接着，主线程调用了event.set()后，event的标志为True，此时event.isSet()为True，所以每个线程开始打印“...is running”。

最后，主线程又sleep 3秒后，主线程调用event.clear()方法使event的标志又变成了False，所以在主线程打印“event is clear”后，其它子线程不再打印“...is running”并退出，整个程序也结束运行。

栅栏对象

栅栏类提供一个简单的同步原语，用于应对固定数量的线程需要彼此相互等待的情况。线程调用 wait() 方法后将阻塞，直到所有线程都调用了 wait() 方法。此时所有线程将被同时释放。

栅栏对象可以被多次使用，但进程的数量不能改变。

In [4]:
# coding=utf-8
import logging
import threading

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [*] %(threadName)s %(message)s"
)
number = 7
threads = []


def open ():
    logging.info("我们三个进去吧----------------------[*]")


class Open_door(threading.Thread):
    def __init__ (self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    
    def run (self):
        try:
            # 重置栅栏为默认的初始态
            # barrier.reset()  # 可以注释，结果不一样
            # 如果屏障不是处于损坏状态
            if not barrier.broken:
                # 返回当前在屏障处等待的线程数
                logging.info(f' 在等着开门——》等待人数(线程)：数量{(barrier.n_waiting + 1)} ')
                # 比如7个线程和三个栅栏，等待中的线程够三个，就通过栅栏，否则2秒后，就按超时异常
                barrier.wait(2)
                logging.info(f' 进入下一关游戏')
        except threading.BrokenBarrierError:
            logging.info("超时等待或退出游戏(发生了超时栅栏破损或重置)")


class Manager(threading.Thread):
    def run (self):
        logging.info('这游戏不开了，关门了！！！')
        barrier.abort()


if __name__ == '__main__':
    barrier = threading.Barrier(3, open)
    logging.info(f"穿过屏障所需的线程数(必须三个人)：{(barrier.parties)}")
    # 创建7个线程--------------------------------------------
    for i in range(number):
        threads.append(Open_door(name='A-%s' % i))
    # 让前面两人重新排队-------------------------------------
    # threads.insert(2, Manager())  # 可以注释，结果不一样 强行暂停游戏
    # 阻塞启动线程-------------------------------------------
    for t in threads:
        t.start()
    for t in threads:
        t.join()

2021-05-24 00:25:40,231 [*] MainThread 穿过屏障所需的线程数(必须三个人)：3
2021-05-24 00:25:40,249 [*] A-0  在等着开门——》等待人数(线程)：数量1 
2021-05-24 00:25:40,256 [*] A-1  在等着开门——》等待人数(线程)：数量1 
2021-05-24 00:25:40,258 [*] A-2  在等着开门——》等待人数(线程)：数量1 
2021-05-24 00:25:40,267 [*] A-2 我们三个进去吧----------------------[*]
2021-05-24 00:25:40,268 [*] A-3  在等着开门——》等待人数(线程)：数量4 
2021-05-24 00:25:40,269 [*] A-2  进入下一关游戏
2021-05-24 00:25:40,269 [*] A-4  在等着开门——》等待人数(线程)：数量1 
2021-05-24 00:25:40,269 [*] A-1  进入下一关游戏
2021-05-24 00:25:40,271 [*] A-0  进入下一关游戏
2021-05-24 00:25:40,274 [*] A-5  在等着开门——》等待人数(线程)：数量1 
2021-05-24 00:25:40,282 [*] A-6  在等着开门——》等待人数(线程)：数量2 
2021-05-24 00:25:40,296 [*] A-5 我们三个进去吧----------------------[*]
2021-05-24 00:25:40,302 [*] A-5  进入下一关游戏
2021-05-24 00:25:40,303 [*] A-4  进入下一关游戏
2021-05-24 00:25:40,303 [*] A-3  进入下一关游戏
2021-05-24 00:25:42,313 [*] A-6 超时等待或退出游戏(发生了超时栅栏破损或重置)


条件对象

条件变量总是与某种类型的锁对象相关联，锁对象可以通过传入获得，或者在缺省的情况下自动创建。当多个条件变量需要共享同一个锁时，传入一个锁很有用。锁是条件对象的一部分，你不必单独地跟踪它。

条件变量（Condition）也是一把锁，除了同步锁的作用外，还具有在线程间通信的功能。

　　有一类线程需要满足条件之后才能够继续执行，Python提供了threading.Condition 对象用于条件变量线程的支持，它除了能提供RLock()或Lock()的方法外，还提供了 wait()、notify()、notifyAll()方法。

lock_con=threading.Condition([Lock/Rlock])： 锁是可选选项，不传人锁，对象自动创建一个RLock()。

wait()：条件不满足时调用，线程会释放锁并进入等待阻塞；

wait_for(predicate, timeout=None)
等待知道条件变量的返回值为True。predicate应该是一个返回值可以解释为布尔值的可调用对象。可以设置timeout以给定最大等待时间。
该方法可以重复调用wait()，直到predicate的返回值解释为True，或发生超时。该方法的返回值就是predicate的最后一个返回值，如果发生超时，返回值为False。
如果忽略超时功能，该方法大致相当于：

while not predicate():
	con.wait()
它与wait()的规则相同：调用前必须先获取锁，阻塞时释放锁，并在被唤醒时重新获取锁并返回。

notify()：条件创造后调用，通知等待池激活一个线程；

notifyAll()：条件创造后调用，通知等待池激活所有线程。

In [1]:
import threading
import time


num = 0
con = threading.Condition()


class Producer(threading.Thread):
    """生产者"""
    def run(self):
        global num
        # 获取锁
        con.acquire()
        while True:
            num += 1
            print('生产了1个，现在有{0}个'.format(num))
            time.sleep(1)
            if num >= 5:
                print('已达到5个，不再生产')
                # 唤醒消费者
                con.notify()
                # 等待-释放锁；被唤醒-获取锁
                con.wait()
        # 释放锁
        con.release()


class Customer(threading.Thread):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.money = 7

    def run(self):
        global num
        while self.money > 0:
            # 由于场景是多个消费者进行抢购，如果将获取锁操作放在循环外(如生产者),
            # 那么一个消费者线程被唤醒时会锁住整个循环，无法实现另一个消费者的抢购。
            # 在循环中添加一套"获取锁-释放锁",一个消费者购买完成后释放锁，其他消费者
            # 就可以获取锁来参与购买。
            con.acquire()
            if num <= 0:
                print('没货了，{0}通知生产者'.format(
                    threading.current_thread().name))
                con.notify()
                con.wait()
            self.money -= 1
            num -= 1
            print('{0}消费了1个, 剩余{1}个'.format(
                threading.current_thread().name, num))
            con.release()
            time.sleep(1)
        print('{0}没钱了-回老家'.format(threading.current_thread().name))


if __name__ == '__main__':
    p = Producer(daemon=True)
    c1 = Customer(name='Customer-1')
    c2 = Customer(name='Customer-2')
    p.start()
    c1.start()
    c2.start()
    c1.join()
    c2.join()

生产了1个，现在有1个
生产了1个，现在有2个
生产了1个，现在有3个
生产了1个，现在有4个
生产了1个，现在有5个
已达到5个，不再生产
Customer-1消费了1个, 剩余4个
Customer-2消费了1个, 剩余3个
Customer-1消费了1个, 剩余2个
Customer-2消费了1个, 剩余1个
Customer-1消费了1个, 剩余0个
没货了，Customer-2通知生产者
生产了1个，现在有1个
生产了1个，现在有2个
生产了1个，现在有3个
生产了1个，现在有4个
生产了1个，现在有5个
已达到5个，不再生产
Customer-1消费了1个, 剩余4个
Customer-2消费了1个, 剩余3个
Customer-1消费了1个, 剩余2个
Customer-2消费了1个, 剩余1个
Customer-2消费了1个, 剩余0个
没货了，Customer-1通知生产者
生产了1个，现在有1个
生产了1个，现在有2个
生产了1个，现在有3个
生产了1个，现在有4个
生产了1个，现在有5个
已达到5个，不再生产
Customer-2消费了1个, 剩余4个
Customer-1消费了1个, 剩余3个
Customer-1消费了1个, 剩余2个
Customer-2消费了1个, 剩余1个
Customer-2没钱了-回老家Customer-1没钱了-回老家



信号量对象